# SeqMF++: Ablation Study

In this notebook we will carry out ablation study experiments in order to investigate impact of low-frequence and high-frequency terms in prediction rule.
$$
    \mathbf{r}_u = \mathbf{Q}\mathbf{p}_u + \mathrm{diag}\Big({\mathbf{S}_u^{(t)}\mathbf{Q}\mathbf{Q}^\top}\Big).
$$
We will stick to the agend presented below in this notebook.

0. Load event log (and prepare it is needed).
1. Fit model on all available data.
2. Split event log to global and local updates (supkey and subkey).
3. Define variations for local model and global one.
4. Instantiate `Federation` from trained model.
5. Define routines for evaluation and monitoring
6. Run simulation with prepared federation and a set of routines.

In [ ]:
import logging

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet

In [ ]:
from collections import defaultdict
from copy import deepcopy
from functools import partial
from itertools import product
from typing import Any

In [ ]:
from scipy.sparse import coo_matrix

In [ ]:
from evaluation import collect_metrics
from seqmf_pp import get_conf_mtx_lap_smooth, get_users_sui, least_squares_P, update_Q_partly

In [ ]:
from fedl.seqmf import SeqMFPlusPlus
from fedl.simulator import BaseGlobalModel, BaseLocalModel, Federation, simulate

In [ ]:
logging.basicConfig(format='%(asctime)s %(levelname)s %(message)s',
                    level=logging.INFO)

## 0 Dataset Preparation

In [ ]:
def assign_session_id(frame: pd.DataFrame, delta: pd.Timedelta):
    mask = frame['timestamp'].diff() > delta
    session_ids = mask.cumsum()
    frame['sid'] = session_ids
    return frame


def read_flatten_lsapp(path: str = 'data/lsapp.tsv'):
    df = pd.read_csv(path, sep='\t', parse_dates=[2]) \
        .query('event_type == "Opened"') \
        .sort_values('timestamp')

    df = (
        df
        .set_index(['user_id', 'timestamp'])
        .sort_index()
        .reset_index(level=[1])
        .groupby(level=[0], group_keys=False)
        .apply(assign_session_id, pd.Timedelta('15min'))
        .reset_index()
        .sort_values('timestamp')
    )

    counter = defaultdict(lambda: len(counter))
    sess_groups = list(
        map(counter.__getitem__,
            df[['user_id', 'sid']].itertuples(index=False, name=None)))
    df.loc[:, 'sess_group'] = sess_groups

    user_groups = ((
        (df['sess_group'].diff().fillna(0) != 0).cumsum() % 9000
    ).diff().fillna(0) < 0).cumsum()
    subkey = user_groups.rename('subkey')
    df = pd.concat([df, subkey], axis=1)
    df['supkey'] = df.subkey // 10

    return df


In [ ]:
df = read_flatten_lsapp('data/lsapp.tsv')
df['iid'] = df.app_name.factorize(True)[0]
df = df.rename(columns={'user_id': 'uid'}) \
    .drop(columns=['session_id', 'app_name', 'event_type', 'sess_group']) \
    .set_index(['supkey', 'subkey', 'uid', 'timestamp']) \
    .sort_index() \
    .reset_index()
df.head()

In [ ]:
sup_period = pd.Timedelta('10d')
sub_period = pd.Timedelta('1d')

## 1 Offline Model

Get some statistics from data.

In [ ]:
n_items = df.iid.max() + 1
n_users = df.uid.max() + 1

In [ ]:
print('Total', n_items, 'items.')
print('Total', n_users, 'users.')

Here, we are going to train model on all availiable data.
This experiment correponds to offline regine when we should get the best base model.

In [ ]:
lap_smooth = 1.0
lr = 5e-05
n_epochs = 6
n_factors = 64
n_steps = 4
pow_bool = False
regularization = 0.1
score_gen_mode = 'sum'
seed = 42

In [ ]:
%%time
rec = SeqMFPlusPlus(
    n_factors=n_factors,
    n_items=n_items,
    n_users=n_users,
    alpha=lap_smooth,
    regularization=regularization,
    learning_rate=lr,
    n_epochs=n_epochs,
    n_steps=n_steps,
    pow_bool=pow_bool,
    random_state=seed,
)
rec.fit(df)

## 2 Global and Local Splits

Global splits are determined by `supkey` and local (specifically, daily) splits are determined by `subkey`.

## 3 Local and Global Models

In [ ]:
class GlobalModel(BaseGlobalModel):
    
    def __init__(self, fed: Federation, alpha, gamma,
                 regularization, learning_rate, n_steps, pow_bool):
        super().__init__(fed)
        self.pow_bool = pow_bool
        self.regularization = regularization
        self.alpha = alpha
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.n_steps = n_steps

    def fit(self, data: pd.DataFrame):
        data_plain = data[['uid', 'iid']] \
            .reset_index()
        
        self.fed.global_factors = update_Q_partly(
            Q=self.fed.global_factors,
            P=self.fed.local_factors,
            data=data_plain,
            n_users=self.fed.nousers,
            n_items=self.fed.noitems,
            user_col='uid',
            item_col='iid',
            pow_bool=self.pow_bool,
            regularization=self.regularization,
            lap_smooth=self.alpha,
            gamma=self.gamma,
            lr=self.learning_rate,
            n_steps=self.n_steps,
        )

In [ ]:
class LocalModel(BaseLocalModel):

    def __init__(self, fed: 'Federation', uid: Any, alpha, gamma,
                 regularization, learning_rate, n_steps, pow_bool):
        super().__init__(fed, uid)
        self.pow_bool = pow_bool
        self.regularization = regularization
        self.alpha = alpha
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.n_steps = n_steps
        
    @property
    def local_factors(self) -> np.ndarray:
        return self.fed.local_factors[self.uid]

    def fit(self, data: pd.DataFrame):
        data_plain = data \
            .reset_index() \
            [['uid', 'iid']]

        Cui, cu = get_conf_mtx_lap_smooth(
            data=data_plain,
            n_subject=self.fed.nousers,
            n_object=self.fed.nousers,
            subject_col_name='uid',
            object_col_name='iid',
            alpha=self.alpha,
            gamma=self.gamma,
        )

        S_dict = get_users_sui(
            data=data_plain,
            n_items=self.fed.noitems,
            user_col='uid',
            item_col='iid',
            pow_bool=self.pow_bool,
            level='uid',
        )

        # regularizer or move it up.
        reg = np.diag(self.regularization * np.ones(self.fed.nofactors))
        res = least_squares_P(C=Cui, cu=cu, S_dict=S_dict,
                              Q=self.fed.global_factors,
                              R=reg, u=self.uid)
        
        self.fed.local_factors[self.uid] = res

    def predict(self, data: pd.DataFrame):
        raise NotImplementedError

In [ ]:
class FixedLocalModel(LocalModel):
    
    def fit(self, data: pd.DataFrame):
        # We just check that update model only on global update.
        #
        # and adjust local and global update periods.
        _, subkey = data.index[0]
        subkey = data.subkey.iloc[0]
        subtime = (subkey + 1) * sub_period
        if subtime % sup_period < sub_period:
            super().fit(data)

In [ ]:
class DummyLocalModel(LocalModel):
    
    def fit(self, data: pd.DataFrame):
        self.fed.local_factors[self.uid] = np.zeros(self.fed.nofactors)
    
    def predict(self, data: pd.DataFrame):
        raise NotImplementedError

In [ ]:
class MarkovianLocalModel(BaseLocalModel):

    def fit(self, data: pd.DataFrame):
        pass

## 4 Instantiate Federation

In [ ]:
def make_federation(rec: SeqMFPlusPlus, data: pd.DataFrame, local_model_ty):
    Cui, cu = get_conf_mtx_lap_smooth(
        data=data,
        n_subject=rec.n_users,
        n_object=rec.n_items,
        subject_col_name='uid',
        object_col_name='iid',
        alpha=rec.alpha,
        gamma=rec.gamma,
    )

    # TODO It seems better to use DataFrame instead of dict.
    #
    # S_dict = pd \
    #     .DataFrame(data=S_dict.items(), columns=['uid', 'S']) \
    #    .set_index(['uid'])
    S_dict = get_users_sui(
        data=data,
        n_items=n_items,
        user_col='uid',
        item_col='iid',
        pow_bool=rec.pow_bool,
        level='uid',
    )
    
    if not isinstance(local_model_ty, BaseLocalModel):
        for uid in S_dict.keys():
            S_dict[uid] = coo_matrix(([], ([], [])), (n_items, n_items)).tocsr()
    
    global_model_factory = partial(GlobalModel, 
                                   alpha=rec.alpha,
                                   gamma=rec.gamma,
                                   regularization=rec.regularization,
                                   learning_rate=rec.learning_rate,
                                   n_steps=rec.n_steps,
                                   pow_bool=rec.pow_bool)
    
    local_model_factory = partial(local_model_ty,
                                  alpha=rec.alpha,
                                  gamma=rec.gamma,
                                  regularization=rec.regularization,
                                  learning_rate=rec.learning_rate,
                                  n_steps=rec.n_steps,
                                  pow_bool=rec.pow_bool)
    
    if local_model_ty == DummyLocalModel:
        local_factors = np.zeros_like(rec.local_factors)
    else:
        rng = np.random.RandomState(42)
        local_factors = rng.normal(0, 1e-3, rec.local_factors.shape)

    return Federation(global_factors=rec.global_factors,
                      local_factors=local_factors,
                      confidence=cu,
                      feedback=Cui,
                      transitions=S_dict,
                      global_model_factory=global_model_factory,
                      local_model_factory=local_model_factory)

## 5 Evaluation and Monitoring

In [ ]:
def estimate_adjacency(data, n_users, n_items):
    """Function estimate_adjacency builds adjacency matrix between all users
    and all items.
    """
    indices = data[['uid', 'iid']].values.T
    values = np.ones(len(data))
    counts = coo_matrix((values, indices), (n_users, n_items))
    colocs = (counts > 0).astype(np.int32)
    return colocs


class Evaluator:

    def __init__(self, adj, topks=(1, 3, 5)):
        self.adj = adj
        self.topks = topks
        self.reports = []

    def __call__(self, keys, frame: pd.DataFrame, fed: Federation):
        if (report := self.evaluate(frame, fed)) is None:
            return
        report['supkey'] = keys[0]
        report['subkey'] = keys[1]
        report = report \
            .reset_index() \
            .set_index(['supkey', 'subkey', 'metric'])
        self.reports.append(report)
    
    @property
    def report(self) -> pd.DataFrame:
        return pd.concat(self.reports)
        
    def evaluate(self, frame: pd.DataFrame, fed: Federation):
        # high-frequency (daily) splitting cuts sessions as well.
        frame = frame \
            .reset_index() \
            .set_index(['uid', 'sid']) \
            .groupby(level=[0, 1]) \
            .filter(lambda x: len(x) >= 2) \
            .reset_index()
        
        if frame.empty:
            return

        # store session as DataFrame of list of lists.
        sessions = frame[['uid', 'sid', 'iid']] \
            .reset_index(drop=True) \
            .set_index(['uid', 'sid']) \
            .groupby(level=[0, 1]) \
            .apply(lambda x: list(x.iid)) \
            .groupby(level=[0]) \
            .apply(lambda x: list(x)) \
            .rename('sessions')

        # restrictions collect_metrics() has. Namely, all computations should
        # be done on a single local model and then lifted to suitable monoid
        # like list. However, this implementation does not allow to do so.
        def predict(*args, **kwargs):
            return self.predict(fed, *args, **kwargs)

        metrics, _ = collect_metrics(predict, sessions, self.adj,
                                     self.topks, 'known_items')

        # Transform dict of dict to list of tuples (k1, k2, v1, v2, ...).
        metrics_data = [(k1, k2) + tuple(v2)
                        for k1, v1 in metrics.items()
                        for k2, v2 in v1.items()]

        # Cast list of tuples to DataFrame of metrics and aggregate values.
        columns = ('uid', 'metric') + self.topks
        records = pd.DataFrame(data=metrics_data, columns=columns)
        report = records \
            .set_index(['metric', 'uid']) \
            .groupby(level=[0]) \
            .mean()

        return report

    def predict(self, fed, uid, sid, sess_items, adj_items):
        Q = fed.global_factors
        P = fed.local_factors
        scores = Q[adj_items] @ (P[uid] + Q[sess_items].sum(axis=0))
        return scores
    
    def reset(self):
        self.reports = []
        return self

In [ ]:
# to manage complexity properly.
class MarkovianEvaluator(Evaluator):
    
    def predict(self, fed, uid, sid, sess_items, adj_items):
        mat = fed.transitions[uid]
        last = sess_items[-1]
        scores = mat[adj_items, last].A[:, 0]
        return scores

## 6 Simulations

In [ ]:
replay = df \
    .set_index(['supkey', 'subkey']) \
    .sort_index()

adj = estimate_adjacency(replay, n_users, n_items)

In [ ]:
reports = {}

In [ ]:
%%time

setups = {
    'baseline': LocalModel,
    'hypothesis1': FixedLocalModel,
    'hypothesis2': DummyLocalModel,
}

evaluator = Evaluator(adj)
evaluator.reset()

for name, local_model_ty in setups.items():
    evaluator.reset()
    fed = make_federation(rec=deepcopy(rec),
                          data=df,
                          local_model_ty=local_model_ty)
    fed = simulate(fed, replay, [evaluator])
    reports[name] = evaluator.report.copy()

In [ ]:
%%time

setups = {
    'item2item': lambda x, y, *args, **kwargs: MarkovianLocalModel(x, y),
}

evaluator = MarkovianEvaluator(adj)
evaluator.reset()

for name, local_model_ty in setups.items():
    evaluator.reset()
    fed = make_federation(rec=deepcopy(rec),
                          data=df,
                          local_model_ty=local_model_ty)
    fed = simulate(fed, replay, [evaluator], full_history=True)
    reports[name] = evaluator.report.copy()

## 7 Visualization

In [ ]:
params = {
 'text.usetex': False,
 'font.family': 'serif',
 'text.latex.preamble': '\\usepackage{times} ',
 'figure.figsize': (5.95, 1.785),
 'figure.constrained_layout.use': True,
 'figure.autolayout': False,
 'font.size': 8,
 'axes.labelsize': 8,
 'legend.fontsize': 6,
 'xtick.labelsize': 6,
 'ytick.labelsize': 6,
 'axes.titlesize': 8,
}

In [ ]:
names = {
    'baseline': 'Full model',
    'hypothesis1': 'Rare local updates',
    'hypothesis2': 'No local updates',
    'item2item': 'SR(od)',
}

In [ ]:
trends = {}
for name in ('baseline', 'hypothesis1', 'hypothesis2', 'item2item'):
    report = reports[name].reset_index(level=[2])
    trends[name] = report[report.metric == 'hr'][5].values

In [ ]:
np.savez('data/dynamic-no-privacy.npz', **trends)

In [ ]:
golden_ratio = 0.6180339887
width = 6.91560069445
height = width * golden_ratio

In [ ]:
with mpl.rc_context(params):
    fig, axes = plt.subplots(1, 2, figsize=(10, 3), dpi=150)

    ax = axes[0]
    for name in ('baseline', 'hypothesis1', 'hypothesis2', 'item2item'):
        ax.plot(trends[name], label=names[name])
    ax.grid()
    ax.legend()
    ax.set_ylim(0.6, 0.9)
    ax.set_xlabel('Cycle')
    ax.set_ylabel('Hit Rate (HR@5)')

    ax = axes[1]
    for name in ('baseline', 'hypothesis1', 'hypothesis2', 'item2item'):
        ax.plot((trends[name] - trends['baseline']).cumsum(), label=names[name])
    ax.grid()
    ax.legend()
    ax.set_ylim(-1.5, 0.1)
    ax.set_xlabel('Cycle')
    ax.set_ylabel(r'Cummulative $\delta\mathrm{HR@5}$')